## Основной код

### Коэффициент изобарного расширения рассчитывается на изобарах как функция температуры.

In [1]:
import numpy as np
from scipy.optimize import curve_fit, least_squares

# Словарь для хранения коэффициентов регрессии. Ключи - значения давления
fit_coefs = {}

# для регрессии плотности посредством минимизации L2-нормы
def rho_Dar_fit(T, a0, a1, m, n):
    return (a0 + a1*T**(1/m))**(1/n)

# для регрессии плотности посредством минимизации L1-нормы
def rho_Dar_fit_l1(coefs, T, y):
    return (coefs[0] + coefs[1]*T**(1/coefs[2]))**(1/coefs[3]) - y

# Функция для получения знаений коэффициентов регрессии
# Вход: словарь {p: {'T': list, 'rho': list}}
# где p - зныение давления; 'T' - лист со значениями температуры; 'rho' - лист со значениями плотности
def fit_data(data):
    
    for p in data.keys():
        
        if len(data[p]['T']) < 8:
            print(f'Skipping P={p}: too few datapoints')
            continue
        
        Ts = np.array(data[p]['T'])
        rhos = np.array(data[p]['rho'])
            
        fit0 = np.polyfit(Ts/Ts.mean(), rhos/rhos.mean(), 1)
        try:
            fit = curve_fit(rho_Dar_fit, Ts/Ts.mean(), rhos/rhos.mean(), p0=(fit0[1], fit0[0], 1, 1))
            fit_coefs[p] = [fit[0][0]*rhos.mean()**fit[0][3], 
                            fit[0][1]*rhos.mean()**fit[0][3]/Ts.mean()**(1/fit[0][2]), fit[0][2], fit[0][3]]
        except:
            try:
                fit = least_squares(rho_Dar_fit_l1, x0=(fit0[1], fit0[0], 1, 1),  
                                    args=(Ts/Ts.mean(), rhos/rhos.mean()), loss='soft_l1')
                fit_coefs[p] = [fit.x[0]*rhos.mean()**fit.x[3], 
                                fit.x[1]*rhos.mean()**fit.x[3]/Ts.mean()**(1/fit.x[2]), fit.x[2], fit.x[3]]
            except:
                print(f'Skipping P={p}: failed fitting')
                continue
                
# Функция для расчета значений коэффициента термического расширения
# перед использованием необходим вызов функции fit_data
def ap_pred(P, T):
    a0, a1, m, n = fit_coefs[P][0], fit_coefs[P][1], fit_coefs[P][2], fit_coefs[P][3]
    return -1/rho_Dar_fit(T, a0, a1, m, n)/n * (a0 + a1*T**(1/m))**(1/n-1) * (a1/m*T**(1/m-1))

## Пример

### $\alpha_p$ жидкого толуола при давлениях 50 и 70 МПа

In [2]:
data = {
    50: {
        'T':   [283.15, 298.15, 308.15, 318.15, 323.15, 333.15, 348.15, 353.15, 373.15, 398.15],
        'rho': [906.60, 894.90, 887.40, 879.70, 875.90, 868.40, 857.20, 853.30, 838.90, 819.60]
    },
    
    70: {
        'T':   [222.51, 248.32, 298.28, 283.15, 298.15, 323.15, 333.15, 348.15, 373.15, 398.15],
        'rho': [963.45, 943.35, 906.88, 916.40, 905.50, 887.60, 880.60, 870.10, 853.10, 835.10]
    }
}

fit_data(data)

# значение коэффициента термического расшиерния при p=50 МПа и T=300 K
print(ap_pred(50, 300))    # K^-1

# значение коэффициента термического расшиерния при p=70 МПа и T=350 K
print(ap_pred(70, 350))    # K^-1

0.0008555213765571343
0.0008091745130009482
